In [ ]:
# Importing pandas library
import pandas as pd
# Reading file 
star_wars_5_df = pd.read_csv("/content/INST737_Star_Wars_IMDB_StarWars5.csv")

In [ ]:
star_wars_5_df.head()

,Unnamed: 0,Review_Date,Author,Rating,Review_Title,Review,Review_Url
0,0,25 April 2016,ivo-cobra8,10.0,"Excellent timeless classic, the best sequel o...",Star Wars: Episode V - The Empire Strikes Back...,/review/rw4592819/?ref_=tt_urv
1,1,27 June 2002,bob the moo,NaN,All the fun of the original with a much bette...,The Rebellion has struck an important blow to ...,/review/rw4592819/?ref_=tt_urv
2,2,7 October 2004,TxMike,NaN,A worthy second (or 5th??) Star Wars installm...,My five children were all pre-teens when 'The ...,/review/rw4592819/?ref_=tt_urv
3,3,26 October 1999,Spleen,10.0,It's NOT the darkest of the trilogy - it's th...,`It avoids having the standard shoot-'em-up en...,/review/rw4592819/?ref_=tt_urv
4,4,16 January 2020,UniqueParticle,10.0,I enjoy this one more than A New Hope\n,So much amazing action and beautiful cinematog...,/review/rw4592819/?ref_=tt_urv


In [ ]:
# Removing unnecessary columns
star_wars_5_df_v2 = star_wars_5_df.drop(columns = ["Review_Date", "Author", "Rating", "Review_Title", "Review_Url", "Unnamed: 0"])

In [ ]:
star_wars_5_df_v2

,Review
0,Star Wars: Episode V - The Empire Strikes Back...
1,The Rebellion has struck an important blow to ...
2,My five children were all pre-teens when 'The ...
3,`It avoids having the standard shoot-'em-up en...
4,So much amazing action and beautiful cinematog...
...,...
1404,This is the one of best movie I have ever seen...
1405,GOOD ACTION AND MOVIE...... THIS MOVIE WORTH T...
1406,Indeed a timeless classic the pioneeri in scie...
1407,This still has to be my favorite Star Wars mov...


In [ ]:
# Topic Modeling Section


In [ ]:
# Installing Topic Modeling Libraries
!pip install regex
!pip install numpy
!pip install gensim
!pip install spacy
!pip install pyLDAvis
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 37.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=08e55e12a5c9abbcc2306a64e286fb4cde71cd450d15be841bdbf76c5d9ac5ac
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wh

In [ ]:
# Importing Necessary Libraries
import regex
import numpy as np
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import nltk
import regex as re
nltk.download('stopwords')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
#Removing emails and newline characters
data = star_wars_5_df_v2['Review'].tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', str(sent)) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", str(sent)) for sent in data]

print(data[0])

Star Wars: Episode V - The Empire Strikes Back (1980) is an excellent timeless classic that it is the best sequel of all time. I love this film to death, it is my second favorite film and the best sequel in the Star Wars franchise that I love. The Empire Strikes Back is a movie that it is one of the best sequels I can think of, it is right there with Aliens (1986), it is right there with Terminator 2: Judgment Day (1991) it is an excellent sequel and it really does establish The Empire as a value treat. Empire Strikes Back is still and excellent film which I really believe is a timeless classic. This is my second favorite film from director Irvin Kershner. Number 1 favorite Irvin Kershner film will be RoboCop 2 (1990) which is very underrated action flick. I still have to review those movies someday.


<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<ipython-input-10-bdb9f04cf91d>:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in data]
<ipython-input-10-bdb9f04cf91d>:8: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', str(sent)) for sent in data]


In [ ]:
# Tokenize words and Clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))

print(data_words[0])

['star', 'wars', 'episode', 'the', 'empire', 'strikes', 'back', 'is', 'an', 'excellent', 'timeless', 'classic', 'that', 'it', 'is', 'the', 'best', 'sequel', 'of', 'all', 'time', 'love', 'this', 'film', 'to', 'death', 'it', 'is', 'my', 'second', 'favorite', 'film', 'and', 'the', 'best', 'sequel', 'in', 'the', 'star', 'wars', 'franchise', 'that', 'love', 'the', 'empire', 'strikes', 'back', 'is', 'movie', 'that', 'it', 'is', 'one', 'of', 'the', 'best', 'sequels', 'can', 'think', 'of', 'it', 'is', 'right', 'there', 'with', 'aliens', 'it', 'is', 'right', 'there', 'with', 'terminator', 'judgment', 'day', 'it', 'is', 'an', 'excellent', 'sequel', 'and', 'it', 'really', 'does', 'establish', 'the', 'empire', 'as', 'value', 'treat', 'empire', 'strikes', 'back', 'is', 'still', 'and', 'excellent', 'film', 'which', 'really', 'believe', 'is', 'timeless', 'classic', 'this', 'is', 'my', 'second', 'favorite', 'film', 'from', 'director', 'irvin', 'kershner', 'number', 'favorite', 'irvin', 'kershner', 'fi

In [ ]:
!python -m spacy download en_core_web_sm


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-11 00:20:30.555615: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 14.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[4]])

['so', 'much', 'amazing', 'action', 'and', 'beautiful', 'cinematography', 'makes', 'for', 'such', 'an', 'enlightening', 'experience', 'in', 'the', 'empire', 'strikes', 'back', 'you', 'know', 'who', 'everyone', 'is', 'which', 'is', 'great', 'plus', 'yoda', 'is', 'introduced', 'love', 'this', 'movie', 'the', 'music', 'is', 'soothing', 'theres', 'romance', 'more', 'of', 'darth_vader', 'and', 'introduces', 'emperor_palpatine', 'what', 'more', 'can', 'you', 'ask', 'for', 'lot', 'to', 'relish', 'and', 'get', 'excited', 'about', 'its', 'such', 'classic', 'gem']


In [ ]:
#Remove Stopwords and Lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

# nlp = spacy.load("en_core_web_sm").load()
# nlp = spacy.load('en', disable=['parser', 'ner'])
# python -m spacy download en_core_web_sm

# nlp = en_core_web_sm.load()
nlp = spacy.load("en_core_web_sm")

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['war', 'empire', 'strike', 'back', 'excellent', 'good', 'sequel', 'time', 'love', 'film', 'death', 'second', 'favorite', 'film', 'good', 'sequel', 'star', 'war', 'franchise', 'love', 'empire', 'strike', 'back', 'movie', 'good', 'sequel', 'think', 'alien', 'excellent', 'sequel', 'really', 'establish', 'empire', 'value', 'empire', 'strike', 'back', 'still', 'excellent', 'film', 'really', 'believe', 'second', 'favorite', 'film', 'number', 'favorite', 'irvin_kershner', 'film', 'underrated', 'action', 'flick', 'still', 'review', 'movie', 'someday']


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 1), (2, 3), (3, 1), (4, 1), (5, 4), (6, 1), (7, 3), (8, 3), (9, 5), (10, 1), (11, 1), (12, 3), (13, 1), (14, 2), (15, 2), (16, 1), (17, 2), (18, 1), (19, 2), (20, 4), (21, 1), (22, 1), (23, 2), (24, 3), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2)]


In [ ]:
!pip install -U gensim
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 20 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 84.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(

[(0, '0.090*"movie" + 0.055*"war" + 0.052*"star" + 0.049*"good" + 0.035*"great" + 0.032*"empire" + 0.025*"strike" + 0.023*"well" + 0.023*"back" + 0.022*"time"'), (1, '0.038*"hate" + 0.028*"fleet" + 0.024*"screen" + 0.019*"agree" + 0.018*"possible" + 0.018*"young" + 0.016*"mention" + 0.014*"depth" + 0.014*"production" + 0.013*"money"'), (2, '0.063*"animation" + 0.063*"computer" + 0.041*"favourite" + 0.023*"hear" + 0.021*"expect" + 0.018*"question" + 0.013*"ill" + 0.012*"accent" + 0.010*"self" + 0.009*"timeless"'), (3, '0.037*"emotional" + 0.032*"sinister" + 0.026*"suspense" + 0.024*"ending" + 0.022*"lose" + 0.022*"conflict" + 0.015*"touch" + 0.013*"cliffhanger" + 0.013*"provide" + 0.012*"anyway"'), (4, '0.037*"style" + 0.034*"learn" + 0.028*"open" + 0.025*"kill" + 0.021*"special_edition" + 0.017*"run" + 0.016*"han_solo" + 0.014*"romantic" + 0.014*"frozen" + 0.013*"teach"'), (5, '0.046*"location" + 0.034*"worth" + 0.030*"friend" + 0.025*"true" + 0.020*"vfx" + 0.018*"spoiler" + 0.015*"cap

/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.update_eta(self.state.get_lambda(), rho)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(

In [ ]:
# Visualize the models
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.374788 -0.016416       1        1  39.878456
14     0.289150  0.038487       2        1  16.081882
10     0.257110 -0.125880       3        1  12.529842
8      0.040144  0.367465       4        1   6.575515
7      0.004645 -0.140549       5        1   4.640410
6     -0.056072 -0.002548       6        1   3.700731
5     -0.073431  0.010525       7        1   2.755005
11    -0.100961  0.014023       8        1   2.260333
1     -0.099450 -0.012813       9        1   2.053342
3     -0.108312 -0.033064      10        1   1.797601
13    -0.096728 -0.020303      11        1   1.752477
4     -0.106315  0.001452      12        1   1.697597
12    -0.104433 -0.024743      13        1   1.569266
2     -0.105855 -0.032156      14        1   1.362829
9     -0.114279 -0.023479      15        1   1.344713, topic_info=            Term         Freq        Total Category  logprob  loglift
15         movie  1968.000000  1968.000000  Default  30.0000  30.0000
9           film  1326.000000  1326.000000  Default  29.0000  29.0000
29           war  1218.000000  1218.000000  Default  28.0000  28.0000
12          good  1078.000000  1078.000000  Default  27.0000  27.0000
22          star  1168.000000  1168.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
3887       crazy     2.545907     3.391756  Topic15  -5.6732   4.0221
3320     firstly     2.474603     3.320461  Topic15  -5.7017   4.0150
1957  excitement     3.518536     6.898848  Topic15  -5.3497   3.6357
359     asteroid     4.998026    18.719120  Topic15  -4.9987   2.9885
107       enough     4.718573    36.198532  Topic15  -5.0562   2.2715

[615 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1062      6  0.891730     absolute
3567     14  0.881399       accent
3438     11  0.917128       accept
241       1  0.978808          act
241       6  0.009788          act
...     ...       ...          ...
3064     13  0.847495  years_later
297       3  0.985083         yoda
849       8  0.362079        young
849       9  0.517255        young
849      14  0.077588        young

[740 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 15, 11, 9, 8, 7, 6, 12, 2, 4, 14, 5, 13, 3, 10])